In [17]:
import numpy as np

def Convolution3D(matrix, fltr, stride=1, padding=0):
    filter_depth, filter_height, filter_width = fltr.shape
    
    result_depth = ((matrix.shape[0] + 2 * padding - filter_depth) // stride) + 1
    result_height = ((matrix.shape[1] + 2 * padding - filter_height) // stride) + 1
    result_width = ((matrix.shape[2] + 2 * padding - filter_width) // stride) + 1
    
    padded_matrix = np.pad(matrix, ((padding, padding), (padding, padding), (padding, padding)), mode='constant')
    result = np.zeros((result_depth, result_height, result_width))
    
    for d in range(result_depth):
        for i in range(result_height):
            for j in range(result_width):
                result[d, i, j] = np.sum(
                    padded_matrix[
                        d*stride:d*stride+filter_depth,
                        i*stride:i*stride+filter_height,
                        j*stride:j*stride+filter_width
                    ] * fltr
                )
            
    return result

In [38]:
from torch.nn.functional import conv3d
from torch import from_numpy
import torch

for s in range(1, 4):
    for p in range(0, 4):
        for i in range(3, 11):
            data = np.round(np.random.rand(i, i, i)*10)/10
            kernel = np.round(np.random.rand(3, 3, 3)*10)/10

            res_matrix = Convolution3D(data, kernel, padding=p, stride=s)
            torch_matrix = conv3d(
                from_numpy(data).reshape(1, 1, i, i, i),
                from_numpy(kernel).reshape(1, 1, 3, 3, 3),
                padding=p,
                stride=s)
            
            res_matrix = np.round(res_matrix, 2)
            torch_matrix = torch.round(torch_matrix[0][0], decimals=2).numpy()
            
            print('Stride:', s)
            print('Padding:', p)
            print('Входящая матрица', data.shape ,':\n', data, '\n')
            print('Ядро', kernel.shape,':\n', kernel, '\n')
            print('Результирующая матрица', res_matrix.shape ,':\n', res_matrix, '\n')
            print('Результирующая матрица torch', torch_matrix.shape ,':\n', torch_matrix, '\n')

            print('Cравниваем результат с torch conv2d:')
            assert (res_matrix == torch_matrix).all()
            print('Успех!', '\n\n')


Stride: 1
Padding: 0
Входящая матрица (3, 3, 3) :
 [[[0.6 0.3 0.9]
  [0.7 0.3 0.9]
  [0.7 0.9 0. ]]

 [[0.9 0.6 0.1]
  [0.  0.3 0.7]
  [0.9 0.3 0.1]]

 [[0.3 0.4 1. ]
  [0.4 0.3 0.4]
  [0.8 0.  0.2]]] 

Ядро (3, 3, 3) :
 [[[1.  0.2 0.2]
  [0.2 0.9 0.5]
  [0.5 0.3 0.5]]

 [[0.4 0.  0.3]
  [0.8 0.1 0.3]
  [0.5 0.7 0.5]]

 [[0.3 0.1 0.3]
  [0.1 0.1 0.8]
  [0.3 0.2 0.1]]] 

Результирующая матрица (1, 1, 1) :
 [[[4.74]]] 

Результирующая матрица torch (1, 1, 1) :
 [[[4.74]]] 

Cравниваем результат с torch conv2d:
Успех! 


Stride: 1
Padding: 0
Входящая матрица (4, 4, 4) :
 [[[0.3 0.9 0.6 0.4]
  [0.9 0.8 0.6 0.5]
  [0.1 0.4 0.3 0.5]
  [0.4 0.8 0.4 0.5]]

 [[0.7 0.1 1.  0.9]
  [0.3 0.1 0.4 0.3]
  [0.5 0.3 0.4 0.8]
  [0.1 0.7 0.8 0.5]]

 [[0.7 0.  0.2 0.9]
  [0.8 0.9 0.9 1. ]
  [0.4 0.7 1.  0.7]
  [0.3 0.1 0.1 0.5]]

 [[0.5 0.5 0.1 0.6]
  [0.2 0.3 0.8 0.8]
  [0.6 0.9 1.  0.9]
  [0.8 0.6 0.3 0.8]]] 

Ядро (3, 3, 3) :
 [[[0.5 0.6 0. ]
  [0.2 0.5 0.7]
  [0.  0.6 0.5]]

 [[0.2 0.8 1. ]
  [0.5 0.7 

 [[[0.28 0.85 1.   ... 1.19 0.64 0.16]
  [0.8  1.65 1.9  ... 1.69 1.03 0.36]
  [0.81 1.45 1.83 ... 2.11 1.01 0.2 ]
  ...
  [0.86 1.75 2.84 ... 2.45 1.66 0.44]
  [0.54 1.09 1.75 ... 1.52 0.54 0.07]
  [0.15 0.38 0.47 ... 0.95 0.65 0.14]]

 [[0.46 1.65 1.8  ... 1.46 0.69 0.12]
  [1.19 2.86 3.42 ... 3.89 2.54 0.74]
  [1.08 2.69 4.19 ... 4.98 2.76 1.33]
  ...
  [1.46 3.61 5.09 ... 4.97 3.35 0.94]
  [1.07 2.53 3.68 ... 3.32 2.2  0.69]
  [0.43 0.51 1.5  ... 2.07 1.47 0.56]]

 [[0.91 2.49 2.77 ... 2.08 1.26 0.5 ]
  [1.4  2.97 4.47 ... 5.16 2.92 1.24]
  [1.43 4.51 6.1  ... 7.11 5.26 1.82]
  ...
  [2.1  6.09 8.13 ... 8.09 4.44 2.15]
  [2.16 4.05 4.92 ... 3.93 3.73 1.07]
  [0.82 1.15 2.96 ... 2.52 1.59 1.11]]

 ...

 [[0.33 1.41 2.33 ... 2.63 1.51 0.53]
  [1.36 2.9  4.23 ... 5.54 3.48 1.33]
  [1.93 4.51 6.94 ... 8.22 5.97 2.39]
  ...
  [2.38 5.87 7.72 ... 6.63 4.53 1.86]
  [1.96 2.96 4.81 ... 2.92 3.08 1.37]
  [0.89 1.47 3.16 ... 3.17 1.97 1.14]]

 [[0.22 0.94 1.69 ... 1.77 1.   0.13]
  [0.73 1.5

Stride: 2
Padding: 3
Входящая матрица (5, 5, 5) :
 [[[0.2 0.4 0.2 0.7 0.2]
  [0.8 0.3 0.  0.8 0.3]
  [1.  0.  0.5 0.5 0.6]
  [0.1 1.  0.2 0.1 0.9]
  [1.  0.5 0.3 0.9 0.7]]

 [[0.5 0.1 0.2 0.4 0.9]
  [0.2 0.2 0.4 0.1 0.9]
  [0.7 0.7 0.4 0.9 0.4]
  [0.8 0.8 0.1 0.6 0.2]
  [0.4 0.6 0.4 0.8 0.3]]

 [[0.3 0.7 0.3 0.3 0.4]
  [0.7 0.2 0.9 0.7 0.5]
  [0.5 0.2 0.3 0.8 0.4]
  [0.6 0.7 0.9 0.2 0.5]
  [0.9 0.7 0.6 0.2 0.4]]

 [[0.6 0.7 0.8 0.6 0. ]
  [0.9 0.1 0.1 0.  0.3]
  [1.  0.6 0.2 0.8 0.6]
  [0.1 0.1 1.  0.7 0.4]
  [0.9 0.9 0.5 0.1 0.6]]

 [[0.6 0.3 0.5 0.1 0.4]
  [0.2 0.7 0.9 1.  0.4]
  [0.6 0.  0.7 0.1 0.7]
  [0.2 0.1 0.3 0.2 0.6]
  [0.9 0.6 0.7 0.1 0.3]]] 

Ядро (3, 3, 3) :
 [[[1.  0.1 0.4]
  [0.7 0.7 0.3]
  [0.2 0.5 0.1]]

 [[0.1 0.2 0.4]
  [0.4 0.9 0.9]
  [0.6 0.9 0.1]]

 [[0.  0.3 1. ]
  [1.  0.9 0.4]
  [0.6 0.7 0.5]]] 

Результирующая матрица (5, 5, 5) :
 [[[0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.